In [1]:
import sys, importlib
# "../" to go back one director
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
import os
# import torch.nn as nn
# from Modules.Utils.Imports import *
# from Modules.Utils.JTNPDESolver import *
# from Modules.Utils.Gradient import Gradient
# from Modules.Utils.ModelWrapper import ModelWrapper
# from Modules.Models.BuildSurfaceFitter import *
# from torch.autograd import Variable
# import Modules.Utils.PDESolver as PDESolver
# import DataFormatter as DF

import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from scipy.io import loadmat
import scipy.io
from scipy import integrate, interpolate
from scipy.sparse import spdiags
from scipy import sparse
import bisect
import time

In [2]:
current_dir = os.getcwd()

def insert(array, n):
    ''' insert n to array and return array and index
    '''
    array_list = list(array)
    
    if n in array_list:
        idx = array_list.index(n)
    else:
        bisect.insort(array_list, n)
        idx = array_list.index(n)
    
    return np.asarray(array_list), idx

def PDE_RHS(t,c,x,D,rho,K,u_data,t_array):
    ''' returns a RHS of the RDE/PDE models
    '''
    # Create the finite difference matrix
    n = len(x)
    e = np.ones((n,))
    A = spdiags([e,-2*e,e], [-1,0,1], n, n)
    A = A.todense()
    A[0,1] = 2
    A[-1,-2] = 2
    dx = x[1] - x[0]
    
    # Compute the aggregated population
    t_new, idx = insert(t_array, t)
    U_new = np.zeros((n,len(t_new)))
    for ix in np.arange(n):
        u_interpolate = interpolate.interp1d(t_array,u_data[ix,:])
        U_new[ix,:] = u_interpolate(t_new)
    u_pop = U_new[:,idx]
    
    c = np.squeeze(c)
    # Compute the rhs
    rhs = D*A.dot(c)/dx**2 + rho*c*(1 - u_pop/K)
    
    return rhs

def PDE_sim(RHS,IC,x,t,D,r,K,u_data):
    ''' run simulation for the RDE/PDE models and return the solutions
    '''
    t_min = np.min(t)
    t_max = np.max(t)
    t_data = t
    #wrapper function for RHS
    def RHS_ty(t,y):
        return RHS(t,y,x,D,r,K,u_data,t_data)
    #Solve 
    sol = integrate.solve_ivp(RHS_ty, t_span=[t_min, t_max], y0=IC, method='RK45', t_eval = t)
    return sol.y

In [3]:
# Load data
path = os.getcwd()+'/NPY_Data/data_wider_2P_E2.npy'
data = np.load(path, allow_pickle=True).item()

x = data['x']
t = data['t']
# Note: we ignored cell density where t>1.4, because almost all cell density approach 1
t = t[:26]
U = data['u_noise']
U = U[:,:26]
# t = t/
X, T = np.meshgrid(x, t, indexing='ij')

shape = U.shape

inputs = np.concatenate([X.reshape(-1)[:, None],
                         T.reshape(-1)[:, None]], axis=1)
outputs = U.reshape(-1)[:, None]

X = inputs[:,0]
T = inputs[:,1]

U = outputs.reshape(shape)

x = np.unique(X)
t = np.unique(T)
u0 = U[:,0]

# PDE function
RHS = PDE_RHS

In [4]:
D_min = 0.0
D_max = 0.12
rho_min = 0.0
rho_max = 12.0

K = 1.0
# Define the maximum numbers of nodes
num_D_nodes = [20]
num_Rho_nodes =  [20]

for num_D_node in num_D_nodes:
    for num_Rho_node in num_Rho_nodes:

        D_vec = np.linspace(D_min, D_max, num_D_node)
        Rho_vec = np.linspace(rho_min, rho_max, num_Rho_node)
        
        # Forward solve for each combination of nodes
        for iD, D in enumerate(D_vec):
            for iRho, rho in enumerate(Rho_vec):
                
                    save_path = 'PrecomputedSolutions_Interp_2P_E2/Sol_'+str(iD+1).zfill(3)+'_'+str(iRho+1).zfill(3)+'.npy'
                    
                    if not os.path.exists(save_path):
                    
                        t0 = time.time()
                        u_sim = PDE_sim(RHS, u0, x, t, D, rho, K, U)
                        elapsed_time = time.time() - t0
                        
                        results = {}
                        results['u_sim'] = u_sim
                        results['shape']   = shape
                        results['x']       = x
                        results['t']       = t
                        results['IC']      = u0
                        results['D']       = D
                        results['rho']     = rho
                        results['K']       = K
                        results['iD']      = iD+1
                        results['iRho']      = iRho+1
                        results['elapsed_time'] = elapsed_time
                        
                        np.save(save_path, results)